# Condemnation Inference
This notebook will hold the code for inferring condemnation on all data

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, load_metric, Dataset
from transformers import TextClassificationPipeline

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import pickle
import pandas as pd
from torch.utils.data import DataLoader
from pymongo import MongoClient
from tqdm import tqdm


## Loading 

In [10]:
client = MongoClient()
db_metoo_tweets = client["new_metoo"]
metoo_tweets = db_metoo_tweets.metoo_tweets

In [11]:
tweet_cursor = metoo_tweets.find({"is_RT":True})

In [48]:
model_path = "./models/fold_1_model.p"
model = torch.load(model_path)
model.eval()
model.to("cuda:0")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

In [51]:
counter = 0
for tweet in tqdm(tweet_cursor):
#     print(tweet)
    if "clean_tweet_masked" not in tweet:
        continue
    else:
        pipe(tweet["clean_tweet_masked"])

0it [00:00, ?it/s]


TypeError: new(): invalid data type 'str'

In [15]:
tweet["clean_tweet_masked"]

'<TWEET>: RT @rorrobracho: Osea, según este diario "se nos cayó el ídolo" porque <TARGET 1> salió del closet y no porque le acusaron de acoso a un…'

In [16]:
tweet["favoritesCount"]

55

In [17]:
tweet["retweetCount"]

71

In [19]:
tweet["time_check"]

{'spacey': {'valid': True,
  'public_date': datetime.datetime(2017, 10, 29, 0, 0)}}

In [20]:
tweet["postedTime"]

datetime.datetime(2017, 10, 31, 17, 2, 35)

In [18]:
tweet.keys()

dict_keys(['_id', 'body', 'postedTime', 'retweetCount', 'favoritesCount', 'quoted_status_id', 'quoted_status_user_id', 'quoted_status_body', 'quoted_status_user_postedTime', 'gnip_url_title', 'gnip_url_description', 'is_RT', 'RT_body', 'RT_user_id', 'RT_id', 'tweet_id', 'user_id', 'RT_target_mentions', 'body_target_mentions', 'body_target_mentions_validated', 'body_target_mentions_validated_true', 'gnip_url_title_mentions', 'lang_pred', 'lang_pred_prob', 'quoted_status_target_mentions', 'clean_targets_n', 'clean_tweet', 'clean_tweet_masked', 'mask_map', 'modified_quote_tweet', 'time_check'])

In [7]:
file = open('./condemnation_predictions.p', 'rb')
preds = pickle.load(file)
file.close()
preds.predictions

array([[-1.005583  ,  0.9191936 ],
       [-0.9339803 ,  0.862211  ],
       [-1.4016373 ,  1.344614  ],
       ...,
       [-0.13647401,  0.00870189],
       [-1.9797113 ,  1.8827175 ],
       [-0.26315764,  0.21105824]], dtype=float32)

In [2]:
file = open('../../data/5_mil_7days_metoo.p', 'rb')
pred_data = pickle.load(file)
file.close()

In [3]:
pred_df = pred_data[["clean_tweet_masked"]].dropna()
pred_df

,clean_tweet_masked
0,"<TWEET>: » “I do love you. . I love you, <TARG..."
4,<TWEET>: <TARGET 1> apologizes for 'aggressive...
5,<TWEET>: No one should have to endure this kin...
6,"<TWEET>: ""New <TARGET 1> sexual assault accusa..."
8,"<TWEET>: Yes this. <TARGET 1>, this clown, the..."
...,...
4683897,<TWEET>: On the one year anniversary of the Ac...
4683898,<TWEET>: <TARGET 1>'s photobombing. The ladies...
4683900,<TWEET>: But but but dude you're a rapist HOW ...
4683901,"<TWEET>: Lisa Bloom, Lawyer Advising <TARGET 1..."


# Trying Trainer

In [25]:
model_path = "./models/fold_1_model.p"
model = torch.load(model_path)
model.eval()
# model.to("cpu")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [34]:
model.to("cpu")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [35]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [36]:
tokenizer(tweet["clean_tweet_masked"])

{'input_ids': [101, 1026, 1056, 28394, 2102, 1028, 1024, 19387, 1030, 20996, 18933, 10024, 9905, 1024, 9808, 5243, 1010, 7367, 12734, 28517, 22939, 9488, 1000, 7367, 16839, 6187, 7677, 3449, 10282, 2080, 1000, 18499, 4226, 1026, 4539, 1015, 1028, 16183, 3695, 3972, 9346, 1061, 2053, 18499, 4226, 3393, 9353, 10383, 4948, 2139, 9353, 19137, 1037, 4895, 1529, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [37]:
from transformers import TextClassificationPipeline

In [38]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

In [39]:
pipe(tweet["clean_tweet_masked"])

[[{'label': 'LABEL_0', 'score': 0.9484457969665527},
  {'label': 'LABEL_1', 'score': 0.051554203033447266}]]

In [5]:
from datasets import Value
pred_dataset = Dataset.from_pandas(pred_df)
pred_dataset = pred_dataset.rename_column("clean_tweet_masked", "text")
new_features = pred_dataset.features.copy()
# new_features["text"] = Value('string')
pred_dataset = pred_dataset.cast(new_features)

  0%|          | 0/280 [00:00<?, ?ba/s]

In [6]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = pred_dataset.map(tokenize_function, batched=True)

  0%|          | 0/2798 [00:00<?, ?ba/s]

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
from transformers import Trainer
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="exp/bart/results",
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=1000,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=1,
)
training_args = TrainingArguments("test-trainer")
training_args.eval_accumulation_steps=1 #pushes predictions out of GPU to mitigate GPU out of memory

trainer = Trainer(
    model,
    args=args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    
)

In [10]:
predictions = trainer.predict(tokenized_datasets[:100])

***** Running Prediction *****
  Num examples = 5
  Batch size = 8


KeyError: 0

In [ ]:
with open('./condemnation_predictions', 'wb') as f:
    pickle.dump(predictions, f)

# Trying Pipeline

In [5]:
import datasets
from transformers import pipeline
from transformers.pipelines.base import *
from tqdm.auto import tqdm

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [9]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [18]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer,  device=0)

In [11]:
pred_df = pred_df.dropna(subset=["clean_tweet_masked"])

In [12]:
pred_df

,clean_tweet_masked
0,"<TWEET>: » “I do love you. . I love you, <TARG..."
4,<TWEET>: <TARGET 1> apologizes for 'aggressive...
5,<TWEET>: No one should have to endure this kin...
6,"<TWEET>: ""New <TARGET 1> sexual assault accusa..."
8,"<TWEET>: Yes this. <TARGET 1>, this clown, the..."
...,...
4683897,<TWEET>: On the one year anniversary of the Ac...
4683898,<TWEET>: <TARGET 1>'s photobombing. The ladies...
4683900,<TWEET>: But but but dude you're a rapist HOW ...
4683901,"<TWEET>: Lisa Bloom, Lawyer Advising <TARGET 1..."


In [23]:
from datasets import Value
pred_dataset = Dataset.from_pandas(pred_df)
pred_dataset = pred_dataset.rename_column("clean_tweet_masked", "text")
new_features = pred_dataset.features.copy()
# new_features["text"] = Value('string')
pred_dataset = pred_dataset.cast(new_features)

  0%|          | 0/280 [00:00<?, ?ba/s]

In [29]:
type(pred_dataset["text"][:10][0])

str

In [ ]:
batch_size=8
for out in tqdm(pipe(pred_dataset["text"]), total=len(pred_dataset)):
    print(out)
    break

In [15]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [16]:
for chunk in chunker(pred_dataset["text"],):
    print(type(chunk))
    break

<class 'list'>


In [26]:
labels = []
i = 1
for chunk in tqdm(chunker(pred_dataset["text"],1)):
    labels.extend(pipe(chunk))
#     torch.cuda.empty_cache()
    i+=1
    if i%100:
        torch.cuda.empty_cache()

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [20]:
pipe(pred_dataset["text"][:32])

[{'label': 'LABEL_1', 'score': 0.8726699948310852},
 {'label': 'LABEL_1', 'score': 0.8576847314834595},
 {'label': 'LABEL_1', 'score': 0.9397012591362},
 {'label': 'LABEL_0', 'score': 0.9323407411575317},
 {'label': 'LABEL_1', 'score': 0.966101348400116},
 {'label': 'LABEL_1', 'score': 0.5745621919631958},
 {'label': 'LABEL_1', 'score': 0.5133087038993835},
 {'label': 'LABEL_1', 'score': 0.5648785829544067},
 {'label': 'LABEL_1', 'score': 0.9608886241912842},
 {'label': 'LABEL_1', 'score': 0.9589126110076904},
 {'label': 'LABEL_1', 'score': 0.9688040018081665},
 {'label': 'LABEL_1', 'score': 0.96010822057724},
 {'label': 'LABEL_0', 'score': 0.789070725440979},
 {'label': 'LABEL_1', 'score': 0.8477528691291809},
 {'label': 'LABEL_0', 'score': 0.5989922285079956},
 {'label': 'LABEL_1', 'score': 0.5745621919631958},
 {'label': 'LABEL_1', 'score': 0.971645712852478},
 {'label': 'LABEL_1', 'score': 0.9548541903495789},
 {'label': 'LABEL_1', 'score': 0.9701395034790039},
 {'label': 'LABEL_0'

In [21]:
pred_dataset["text"][:32]

['<TWEET>: » “I do love you. . I love you, <TARGET 1>. .”',
 "<TWEET>: <TARGET 1> apologizes for 'aggressive and crude' behavior toward women… URL bench #Lifestyle",
 '<TWEET>: No one should have to endure this kind of torture... Incredibly brave of @AnnabellSciorra to share her story of <TARGET 1>’s abuse URL\n\n<QUOTED TWEET>: Sciorra was still living in fear of <TARGET 1>, she said, and slept with a baseball bat by her bed. URL',
 '<TWEET>: "New <TARGET 1> sexual assault accusations emerge" via FOX NEWS URL URL',
 '<TWEET>: Yes this. <TARGET 1>, this clown, the entire frat house at Fox. How could we expect first female presidential candidate to get a fair shake?',
 '<TWEET>: Over 300 Claims Against <TARGET 1>! URL',
 '<TWEET>: Over 300 Claims Against <TARGET 1>! URL URL',
 '<TWEET>: Rose McGowan accuses <TARGET 1> of Rape - News Today - Breaking News URL via @YouTube',
 '<TWEET>: <TARGET 1>, Hefner and the Poor Excuse that Explains a Lot via @NYTimes URL',
 '<TWEET>: Ellen sexual ha

0it [00:00, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 7.80 GiB total capacity; 597.34 MiB already allocated; 9.56 MiB free; 660.00 MiB reserved in total by PyTorch)

In [27]:
dataset = datasets.load_dataset("imdb", name="plain_text", split="unsupervised")

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /home/geev/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


['<TWEET>: » “I do love you. . I love you, <TARGET 1>. .”',
 "<TWEET>: <TARGET 1> apologizes for 'aggressive and crude' behavior toward women… URL bench #Lifestyle",
 '<TWEET>: No one should have to endure this kind of torture... Incredibly brave of @AnnabellSciorra to share her story of <TARGET 1>’s abuse URL\n\n<QUOTED TWEET>: Sciorra was still living in fear of <TARGET 1>, she said, and slept with a baseball bat by her bed. URL',
 '<TWEET>: "New <TARGET 1> sexual assault accusations emerge" via FOX NEWS URL URL',
 '<TWEET>: Yes this. <TARGET 1>, this clown, the entire frat house at Fox. How could we expect first female presidential candidate to get a fair shake?',
 '<TWEET>: Over 300 Claims Against <TARGET 1>! URL',
 '<TWEET>: Over 300 Claims Against <TARGET 1>! URL URL',
 '<TWEET>: Rose McGowan accuses <TARGET 1> of Rape - News Today - Breaking News URL via @YouTube',
 '<TWEET>: <TARGET 1>, Hefner and the Poor Excuse that Explains a Lot via @NYTimes URL',
 '<TWEET>: Ellen sexual ha

In [29]:
dataset[0]

{'text': 'I admit, the great majority of films released before say 1933 are just not for me. Of the dozen or so "major" silents I have viewed, one I loved (The Crowd), and two were very good (The Last Command and City Lights, that latter Chaplin circa 1931).<br /><br />So I was apprehensive about this one, and humor is often difficult to appreciate (uh, enjoy) decades later. I did like the lead actors, but thought little of the film.<br /><br />One intriguing sequence. Early on, the guys are supposed to get "de-loused" and for about three minutes, fully dressed, do some schtick. In the background, perhaps three dozen men pass by, all naked, white and black (WWI ?), and for most, their butts, part or full backside, are shown. Was this an early variation of beefcake courtesy of Howard Hughes?',
 'label': -1}

In [26]:
pred_dataset

Dataset({
    features: ['tweet_id', 'user_id', 'text', 'body_target_mentions_validated', '__index_level_0__'],
    num_rows: 2061369
})

In [15]:
from torch.utils.data import Dataset
class MyDataSet(Dataset):
    def __init__ (self, df, X_col):
        '''
        So that we can move the entire dataset to the GPU.
        :param X: float32 data scaled numpy array
        :param y: float32 data scaled numpy vector
        :param device: 'cpu' or 'cuda:0'
        '''
        self.X = list(df[X_col])
        # y vector needs to be in a column vector (or at least it
        # did in the normal dataset.)
#         self.y = torch.from_numpy(df[y_col])

    def __len__(self):
        return list(self.X.size())[0]

    def __getitem__(self, item):
        return self.X[item]

In [16]:
X_col = "clean_tweet_masked"

In [17]:
pred_df[X_col]

0          <TWEET>: » “I do love you. . I love you, <TARG...
4          <TWEET>: <TARGET 1> apologizes for 'aggressive...
5          <TWEET>: No one should have to endure this kin...
6          <TWEET>: "New <TARGET 1> sexual assault accusa...
8          <TWEET>: Yes this. <TARGET 1>, this clown, the...
                                 ...                        
4683894    <TWEET>: Democratic National Committee to Give...
4683895    <TWEET>: We are also tired of Hollywood; they ...
4683898    <TWEET>: <TARGET 1>'s photobombing. The ladies...
4683901    <TWEET>: Lisa Bloom, Lawyer Advising <TARGET 1...
4683902    <TWEET>: Don Jr. trolls silent Hillary over <T...
Name: clean_tweet_masked, Length: 2061369, dtype: object

In [18]:
pred_dataset = MyDataSet(pred_df, "clean_tweet_masked")

In [19]:
loader = DataLoader(pred_dataset,batch_size=64,shuffle=False, num_workers=1)

In [20]:
for X in loader:
    print(X)
    print("-------")

AttributeError: 'list' object has no attribute 'size'

In [54]:
pred_data.columns

Index(['_id', 'body', 'postedTime', 'retweetCount', 'favoritesCount',
       'quoted_status_id', 'quoted_status_user_id', 'quoted_status_body',
       'quoted_status_user_postedTime', 'gnip_url_title',
       'gnip_url_description', 'is_RT', 'RT_body', 'RT_user_id', 'RT_id',
       'tweet_id', 'user_id', 'RT_target_mentions', 'body_target_mentions',
       'gnip_url_title_mentions', 'lang_pred', 'lang_pred_prob',
       'quoted_status_target_mentions', 'RT_target_mentions_metoo',
       'all_fields_n_targets', 'body_target_mentions_metoo',
       'body_target_mentions_n_targets', 'body_target_mentions_target',
       'gnip_url_title_mentions_metoo', 'quoted_status_target_mentions_metoo',
       'masked_body', 'propn_masked_body', 'body_target_mentions_validated',
       'body_target_mentions_validated_true', 'clean_targets_n', 'clean_tweet',
       'clean_tweet_masked', 'mask_map', 'modified_quote_tweet', 'time_check',
       'all_target_mentions_metoo', 'gnip_url_title_mentions_n_targ

In [ ]:
pipe()

In [55]:
cleaned_masked_texts = pred_data.clean_tweet_masked
texts

0          <TWEET>: » “I do love you. . I love you, <TARG...
1                                                        NaN
2                                                        NaN
3                                                        NaN
4          <TWEET>: <TARGET 1> apologizes for 'aggressive...
                                 ...                        
4683900    <TWEET>: But but but dude you're a rapist HOW ...
4683901    <TWEET>: Lisa Bloom, Lawyer Advising <TARGET 1...
4683902    <TWEET>: Don Jr. trolls silent Hillary over <T...
4683903                                                  NaN
4683904                                                  NaN
Name: clean_tweet_masked, Length: 4683905, dtype: object

In [56]:
pred_dataset = Dataset.from_pandas(pred_df)

In [57]:
pred_dataset

Dataset({
    features: ['tweet_id', 'user_id', 'clean_tweet_masked', 'body_target_mentions_validated', '__index_level_0__'],
    num_rows: 2061369
})

In [66]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
    return tokenizer(example["clean_tweet_masked"], truncation=True)

tokenized_datasets = pred_dataset.map(tokenize_function, batched=True)


  0%|          | 0/2062 [00:00<?, ?ba/s]

AttributeError: 'DataCollatorWithPadding' object has no attribute 'cpu'

In [67]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Loading the model

In [68]:
tokenized_datasets

Dataset({
    features: ['__index_level_0__', 'attention_mask', 'body_target_mentions_validated', 'clean_tweet_masked', 'input_ids', 'token_type_ids', 'tweet_id', 'user_id'],
    num_rows: 2061369
})

In [21]:
model_path = "./sample_model.p"
model = torch.load(model_path)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [47]:
tokenized_datasets

Dataset({
    features: ['__index_level_0__', 'attention_mask', 'body_target_mentions_validated', 'input_ids', 'token_type_ids', 'tweet_id', 'user_id'],
    num_rows: 2061369
})

In [48]:
tokenized_datasets = tokenized_datasets.remove_columns(["body_target_mentions_validated"])
# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [49]:
test_dataloader = DataLoader(tokenized_datasets, shuffle=False, batch_size=8)

In [50]:
for batch in test_dataloader:
    print(batch)
    break

TypeError: new(): invalid data type 'numpy.str_'